In [13]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

[nltk_data] Downloading package stopwords to /Users/oanh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/oanh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
data = pd.read_csv('Dataset_Suicide.csv', lineterminator = '\n')
data

,Unnamed: 0,Post,Suicidal_label,Sentiment_label
0,0,Ex Wife Threatening SuicideRecently I left my ...,0,0
1,1,Am I weird I don t get affected by compliments...,1,1
2,2,Finally is almost over So I can never hear ...,1,0
3,3,i need helpjust help me im crying so hard,0,0
4,4,I m so lostHello my name is Adam and I ve b...,0,0
...,...,...,...,...
226948,227680,I sound like a dudebro but I can t handle my f...,0,0
226949,227681,Fuck my sister She is such I fucking bitch and...,1,0
226950,227682,I ve been suicidal for years and no one knowsT...,0,1
226951,227683,My boyfriend is sick so I took some Polaroids ...,1,0


In [15]:
import pandas as pd

# Assuming you have a DataFrame 'df' with the 'label' column
data['Suicidal_label'] = data['Suicidal_label'].map({1: 0, 0: 1})
data['Sentiment_label'] = data['Sentiment_label'].map({1: 2, 0: 1, 2: 0})
data.head()

,Unnamed: 0,Post,Suicidal_label,Sentiment_label
0,0,Ex Wife Threatening SuicideRecently I left my ...,1,1
1,1,Am I weird I don t get affected by compliments...,0,2
2,2,Finally is almost over So I can never hear ...,0,1
3,3,i need helpjust help me im crying so hard,1,1
4,4,I m so lostHello my name is Adam and I ve b...,1,1


In [16]:
print(data.shape)
data['Suicidal_label'].value_counts()

(226953, 4)


1    113534
0    113419
Name: Suicidal_label, dtype: int64

In [17]:
data['Post'].fillna('default_text', inplace=True)
def preprocess(string):
    phrase = str(string)
    phrase = re.sub('[^a-z]+', ' ', phrase, flags = re.IGNORECASE)
    phrase = re.sub('(\s+)', ' ', phrase)
    phrase = re.sub('http\S+', ' ', phrase)
    phrase = phrase.lower()
    words_li = ['filler']
    li = list(stopwords.words()) + words_li
    text_tokens = word_tokenize(phrase)
    return " ".join([word for word in text_tokens if word not in li])
print(data['Post'].apply(lambda x: len(x.split(' '))).sum())

34592572


In [18]:
data['Post'] = data['Post'].map(lambda string: preprocess(string))
print(data['Post'].apply(lambda x: len(x.split(' '))).sum())

9211270


In [19]:
X = data['Post'].values
y = data['Suicidal_label'].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.8, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.75, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(136171,) (45391,) (45391,)


In [20]:
import gensim
import gensim.downloader as api 
import numpy as np
import nltk

# Load FastText model (ensure you have FastText installed and the model is downloaded)
wv = api.load("fasttext-wiki-news-subwords-300")

In [21]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.key_to_index:
            mean.append(wv.get_vector(word))
            all_words.add(wv.key_to_index[word])

    if not mean:
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [22]:
def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list])

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(str(text), language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            word = word.lower()
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [23]:
# Process training data
train_X_tokenized = [w2v_tokenize_text(text) for text in train_X]
X_train_word_average = word_averaging_list(wv, train_X_tokenized)
# Process validation data
val_X_tokenized = [w2v_tokenize_text(text) for text in val_X]
X_val_word_average = word_averaging_list(wv, val_X_tokenized)
# Process testing data
test_X_tokenized = [w2v_tokenize_text(text) for text in test_X]
X_test_word_average = word_averaging_list(wv, test_X_tokenized)

In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score

# Create the KNN model
knn = KNeighborsClassifier()

# Define the parameters for GridSearch CV
knn_params = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance']
}

# Perform GridSearch CV to find the best hyperparameters
grid_search = GridSearchCV(estimator=knn, param_grid=knn_params, cv=5, scoring='accuracy', n_jobs=-1)
grid_result = grid_search.fit(X_train_word_average, train_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.818911 using {'n_neighbors': 10, 'weights': 'uniform'}


In [25]:
# Get the best KNN model and best parameters
best_knn_model = grid_search.best_estimator_
best_knn_params = grid_search.best_params_

# Perform cross-validation using the best model
cv_scores_accuracy = cross_val_score(best_knn_model, X_train_word_average, train_y, cv=5, scoring='accuracy')
cv_scores_recall = cross_val_score(best_knn_model, X_train_word_average, train_y, cv=5, scoring='recall_micro')

# Print average cross-validation scores for accuracy and recall
print(f'Cross-Validation Accuracy: {100 * np.mean(cv_scores_accuracy):.2f}%')
print(f'Cross-Validation Recall: {100 * np.mean(cv_scores_recall):.2f}%')

# Fit the best model on the training data and make predictions
best_knn_model.fit(X_train_word_average, train_y)
y_train_pred = best_knn_model.predict(X_train_word_average)
y_test_pred = best_knn_model.predict(X_test_word_average)

# Calculate and print the train and test accuracy and recall scores
print('Train accuracy score:', accuracy_score(train_y, y_train_pred))
print('Test accuracy score:', accuracy_score(test_y, y_test_pred))
print('Train recall score:', recall_score(train_y, y_train_pred, average='micro'))
print('Test recall score:', recall_score(test_y, y_test_pred, average='micro'))

# Print the classification report
print(classification_report(test_y, y_test_pred))

Cross-Validation Accuracy: 81.89%
Cross-Validation Recall: 81.89%
Train accuracy score: 0.8447687099308957
Test accuracy score: 0.8234231455574894
Train recall score: 0.8447687099308957
Test recall score: 0.8234231455574894
              precision    recall  f1-score   support

           0       0.94      0.69      0.80     22684
           1       0.75      0.96      0.84     22707

    accuracy                           0.82     45391
   macro avg       0.85      0.82      0.82     45391
weighted avg       0.85      0.82      0.82     45391



In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score

# Create the Logistic Regression model
logreg = LogisticRegression(n_jobs=1)

# Define the parameters for GridSearch CV
logreg_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Perform GridSearch CV to find the best hyperparameters
grid_search = GridSearchCV(estimator=logreg, param_grid=logreg_params, cv=5, scoring='accuracy', n_jobs=-1)
grid_result = grid_search.fit(X_train_word_average, train_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Users/oanh/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/oanh/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

Best: 0.896902 using {'C': 10, 'solver': 'sag'}


In [27]:
# Get the best Logistic Regression model and best parameters
best_logreg_model = grid_search.best_estimator_
best_logreg_params = grid_search.best_params_

# Perform cross-validation using the best model
cv_scores_accuracy = cross_val_score(best_logreg_model, X_train_word_average, train_y, cv=5, scoring='accuracy')
cv_scores_recall = cross_val_score(best_logreg_model, X_train_word_average, train_y, cv=5, scoring='recall_micro')

# Print average cross-validation scores for accuracy and recall
print(f'Cross-Validation Accuracy: {100 * np.mean(cv_scores_accuracy):.2f}%')
print(f'Cross-Validation Recall: {100 * np.mean(cv_scores_recall):.2f}%')

# Fit the best model on the training data and make predictions
best_logreg_model.fit(X_train_word_average, train_y)
y_train_pred = best_logreg_model.predict(X_train_word_average)
y_test_pred = best_logreg_model.predict(X_test_word_average)

# Calculate and print the train and test accuracy and recall scores
print('Train accuracy score:', accuracy_score(train_y, y_train_pred))
print('Test accuracy score:', accuracy_score(test_y, y_test_pred))
print('Train recall score:', recall_score(train_y, y_train_pred, average='micro'))
print('Test recall score:', recall_score(test_y, y_test_pred, average='micro'))

# Print the classification report
print(classification_report(test_y, y_test_pred))

Cross-Validation Accuracy: 89.69%
Cross-Validation Recall: 89.69%
Train accuracy score: 0.8979371525508368
Test accuracy score: 0.8982838007534534
Train recall score: 0.8979371525508368
Test recall score: 0.8982838007534534
              precision    recall  f1-score   support

           0       0.91      0.88      0.90     22684
           1       0.89      0.91      0.90     22707

    accuracy                           0.90     45391
   macro avg       0.90      0.90      0.90     45391
weighted avg       0.90      0.90      0.90     45391



In [38]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import recall_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score

# Create the SGD Classifier model
sgd_clf = SGDClassifier()

# Define the parameters for GridSearch CV
sgd_params = {
    'alpha': [0.000001, 0.00001, 0.0001, 0.001, 0.01],
    'penalty': ['l2', 'l1', 'elasticnet']
}

# Perform GridSearch CV to find the best hyperparameters
grid_search = GridSearchCV(estimator=sgd_clf, param_grid=sgd_params, cv=5, scoring='accuracy', n_jobs=-1)
grid_result = grid_search.fit(X_train_word_average, train_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.897320 using {'alpha': 1e-05, 'penalty': 'l2'}


In [29]:
# Get the best SGD Classifier model and best parameters
best_sgd_model = grid_search.best_estimator_
best_sgd_params = grid_search.best_params_

# Perform cross-validation using the best model
cv_scores_accuracy = cross_val_score(best_sgd_model, X_train_word_average, train_y, cv=5, scoring='accuracy')
cv_scores_recall = cross_val_score(best_sgd_model, X_train_word_average, train_y, cv=5, scoring='recall_micro')

# Print average cross-validation scores for accuracy and recall
print(f'Cross-Validation Accuracy: {100 * np.mean(cv_scores_accuracy):.2f}%')
print(f'Cross-Validation Recall: {100 * np.mean(cv_scores_recall):.2f}%')

# Fit the best model on the training data and make predictions
best_sgd_model.fit(X_train_word_average, train_y)
y_train_pred = best_sgd_model.predict(X_train_word_average)
y_test_pred = best_sgd_model.predict(X_test_word_average)

# Calculate and print the train and test accuracy and recall scores
print('Train accuracy score:', accuracy_score(train_y, y_train_pred))
print('Test accuracy score:', accuracy_score(test_y, y_test_pred))
print('Train recall score:', recall_score(train_y, y_train_pred, average='micro'))
print('Test recall score:', recall_score(test_y, y_test_pred, average='micro'))

# Print the classification report
print(classification_report(test_y, y_test_pred))

Cross-Validation Accuracy: 89.69%
Cross-Validation Recall: 89.33%
Train accuracy score: 0.8863340946310154
Test accuracy score: 0.8864752924588575
Train recall score: 0.8863340946310154
Test recall score: 0.8864752924588575
              precision    recall  f1-score   support

           0       0.94      0.83      0.88     22684
           1       0.85      0.94      0.89     22707

    accuracy                           0.89     45391
   macro avg       0.89      0.89      0.89     45391
weighted avg       0.89      0.89      0.89     45391



In [32]:
import re
import nltk
import numpy as np
import gensim.downloader as api
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping

# Assuming word embeddings size is 300 for FastText model
embedding_dimension = 300

# Ensure that X_train_word_average has the correct shape
X_train_word_average = word_averaging_list(wv, train_X_tokenized)
X_train_word_average = np.array(X_train_word_average)
X_train_word_average = X_train_word_average.reshape(X_train_word_average.shape[0], embedding_dimension)

# Ensure that train_y has the correct shape
train_y = np.array(train_y)

# Build the Recursive Neural Network (RNN) model
model = Sequential()
model.add(Dense(64, input_shape=(300,)))  # Assuming word embeddings size is 300 for FastText model
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, hence 1 neuron with sigmoid activation

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True)

# Train the model
history = model.fit(X_train_word_average, train_y, batch_size=32, epochs=100, 
                    verbose=1, callbacks=[early_stopping], validation_data=(X_test_word_average, test_y))

# Evaluate the model on the test set
score = model.evaluate(X_test_word_average, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
4256/4256 [==============================] - 2s 440us/step - loss: 0.3227 - accuracy: 0.8724 - val_loss: 0.2706 - val_accuracy: 0.8942
Epoch 2/100
4256/4256 [==============================] - 2s 431us/step - loss: 0.2758 - accuracy: 0.8931 - val_loss: 0.2519 - val_accuracy: 0.9011
Epoch 3/100
4256/4256 [==============================] - 2s 421us/step - loss: 0.2629 - accuracy: 0.8986 - val_loss: 0.2479 - val_accuracy: 0.9054
Epoch 4/100
4256/4256 [==============================] - 2s 424us/step - loss: 0.2559 - accuracy: 0.9016 - val_loss: 0.2428 - val_accuracy: 0.9048
Epoch 5/100
4256/4256 [==============================] - 2s 426us/step - loss: 0.2507 - accuracy: 0.9037 - val_loss: 0.2366 - val_accuracy: 0.9072
Epoch 6/100
4256/4256 [==============================] - 2s 423us/step - loss: 0.2479 - accuracy: 0.9043 - val_loss: 0.2341 - val_accuracy: 0.9085
Epoch 7/100
4256/4256 [==============================] - 2s 422us/step - loss: 0.2447 - accuracy: 0.9058 - val_loss: 0

In [33]:
# Make predictions on the training and testing data
y_train_pred = model.predict(X_train_word_average)
y_train_pred = (y_train_pred > 0.5).astype(int)

y_test_pred = model.predict(X_test_word_average)
y_test_pred = (y_test_pred > 0.5).astype(int)

# Print the accuracy and recall scores for both training and testing data
from sklearn.metrics import accuracy_score, recall_score, classification_report

print('Train accuracy score:', accuracy_score(train_y, y_train_pred))
print('Test accuracy score:', accuracy_score(test_y, y_test_pred))
print('Train recall score:', recall_score(train_y, y_train_pred, average='micro'))
print('Test recall score:', recall_score(test_y, y_test_pred, average='micro'))

# Print the classification report for the testing data
print(classification_report(test_y, y_test_pred))

1419/1419 [==============================] - 0s 198us/step
Train accuracy score: 0.9251162141718868
Test accuracy score: 0.9171421647463154
Train recall score: 0.9251162141718868
Test recall score: 0.9171421647463154
              precision    recall  f1-score   support

           0       0.92      0.91      0.92     22684
           1       0.91      0.92      0.92     22707

    accuracy                           0.92     45391
   macro avg       0.92      0.92      0.92     45391
weighted avg       0.92      0.92      0.92     45391



In [34]:
# New texts to predict
new_texts = ['I am tired of my life i want to end my life', 
             'I have a nice day',
             "I went to the grocery store this morning and they had a fruit that I'd never seen before. I asked an employee what it was and she didn't know so we tried to Google it but nothing came up! At that point I as too curious to give up, so bought it and said I'd figure it out later. They didn't even know how to charge me so it only cost $1! I'm gonna try it later, maybe do a cheese and charcuterie board with some wine and be fancy for no reason lol.",
             "I'm autistic and I've had enough",
             "Ok I've had enough and I don't want to be autistic anymore",
            'I think that this is goodbye',
            'Someone Fucking Help Me',
            'I don’t know why I’m so sad',
             "But I also haven’t been able to shake this feeling for the past couple of months that I just want to fucking explode. I want to go off the deep end and just throw everything I have away. I want to hurt the people I love so that one day they will throw me away too. It’s like everyday I do something good for myself is just me putting off the inevitable crash and burn. I just don’t want to give a fuck anymore and I don’t know why when everything has been going so well.",
             "Like, I don’t know. I just don’t know. I’m pretty much on the verge of tears over nothing. There might be something, but I guess my brain just isn’t showing me. I just lay in my room and feel lonely and depressed for hours on end. It gets repetitive sometimes. I’m sick of this feeling.",
            "I'm going to explode",
            "my husband is terminally i’ll and is constantly in pain for over 2.5 years now I can’t take it anymore",
            "I am 54 with complex mental health and physical health issues none of them terminal  (at least in the books of law). So I have no access to VAD. I have no want to be in this world any longer - I cannot live in my own head and nor with the physical pain I am in with back, shoulder, migraine issues ..",
            "I'm 58 and I can't do this anymore. Everyday is misery. I live with a woman who uses me fir money and my prescriptions. So I as in horrendous pain most of the time. I'm always at risk of going to jail if i say no to her. She takes every cent i have . I have no where to go but where I am going tonight. If anyone reads this thanks for taking the time. I was here.",
            "I love my life"]
# Preprocess the new texts
new_texts_preprocessed = [preprocess(text) for text in new_texts]

new_texts_tokenized = [w2v_tokenize_text(text) for text in new_texts_preprocessed]

# Generate word embeddings for new texts
new_word_average = word_averaging_list(wv, new_texts_tokenized)
new_word_average = new_word_average.reshape(-1, wv.vector_size)

# Predict labels for new data
new_predictions = model.predict(new_word_average)
new_predictions = np.round(new_predictions).flatten()

# Convert new_predictions to an array of integers
new_predictions = new_predictions.astype(int)

# Inverse transform the predictions to get original labels
new_predictions_labels = le.inverse_transform(new_predictions)

# Print the predicted labels for the new texts
for text, label in zip(new_texts, new_predictions_labels):
    print(f'Text: {text}\nPredicted Label: {label}\n')

1/1 [==============================] - 0s 6ms/step
Text: I am tired of my life i want to end my life
Predicted Label: 1

Text: I have a nice day
Predicted Label: 0

Text: I went to the grocery store this morning and they had a fruit that I'd never seen before. I asked an employee what it was and she didn't know so we tried to Google it but nothing came up! At that point I as too curious to give up, so bought it and said I'd figure it out later. They didn't even know how to charge me so it only cost $1! I'm gonna try it later, maybe do a cheese and charcuterie board with some wine and be fancy for no reason lol.
Predicted Label: 0

Text: I'm autistic and I've had enough
Predicted Label: 0

Text: Ok I've had enough and I don't want to be autistic anymore
Predicted Label: 1

Text: I think that this is goodbye
Predicted Label: 1

Text: Someone Fucking Help Me
Predicted Label: 1

Text: I don’t know why I’m so sad
Predicted Label: 0

Text: But I also haven’t been able to shake this feeling f